In [1]:
!python -m pip install git+https://github.com/huggingface/transformers
! pip install openpyxl

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ckn525es
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ckn525es
  Resolved https://github.com/huggingface/transformers to commit b54109c7466f6e680156fbd30fa929e2e222d730
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 58.4 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.46.0.dev0-py3-none-any.whl size=9970034 sha256=4820cd0ffb1ac063802fc42d2662a3bee69dd3e3cd65dfc15a14b0b0d0a1b2bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-aetzaj71/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully u

In [2]:
import re
import openpyxl
import cv2  # Import OpenCV for real-time capture
from PIL import Image
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
import numpy as np
from google.colab import files

In [3]:
# Load the Qwen2 model and processor
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

# Create a new Excel workbook
workbook = openpyxl.Workbook()
sheet = workbook.active
sheet.title = "Product Analysis"

# Add headers
headers = ["Product Name", "Category", "Quantity", "Count", "Expiry Date", "Freshness Index", "Shelf Life"]
sheet.append(headers)

# Regular expression patterns to extract data
packaged_product_pattern = r"Product Name: (.*)\n  - Product Category: (.*)\n  - Product Quantity: (.*)\n  - Product Count: (.*)\n  - Expiry Date: (.*)"
fruits_vegetables_pattern = r"Type of fruit/vegetable: (.*)\n  - Freshness Index: (.*)\n  - Estimated Shelf Life: (.*)"


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [5]:
# Let user upload the image
uploaded = files.upload()

# Iterate over all uploaded files
for image_name in uploaded.keys():
    # Open the image using PIL
    image = Image.open(image_name)
    image = image.resize((512, 512))

    # Prepare the text prompt for predicting product details and freshness
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "image",
                    "image_url": "Captured from webcam"  # Description for internal use
                },
                {
                    "type": "text",
                    "text": """This image contains fruits, vegetables, or packaged products.
                    Please analyze the image and provide:
                    - For packaged products:
                        - Product Name
                        - Product Category
                        - Product Quantity
                        - Product Count
                        - Expiry Date (if available)
                    - For fruits and vegetables:
                        - Type of fruit/vegetable
                        - Freshness Index (based on visual cues)
                        - Estimated Shelf Life"""
                }
            ]
        }
    ]

    # Prepare the text prompt for processing
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    # Process the image and prompt for model input
    inputs = processor(
        text=[text_prompt],
        images=[image],
        padding=True,
        return_tensors="pt"
    )

    # Move inputs to GPU if available
    inputs = inputs.to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate output from the model
    output_ids = model.generate(**inputs, max_new_tokens=1024)

    # Decode the generated output
    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )[0]

    # Print the output text for the current image
    print(f"Analysis Output for the captured frame:")
    print(output_text)

    # Extract packaged product information
    packaged_product_match = re.search(packaged_product_pattern, output_text)
    fruits_vegetables_match = re.search(fruits_vegetables_pattern, output_text)

    if packaged_product_match:
        product_name = packaged_product_match.group(1).strip()
        category = packaged_product_match.group(2).strip()
        quantity = packaged_product_match.group(3).strip()
        count = packaged_product_match.group(4).strip()
        expiry_date = packaged_product_match.group(5).strip()
    else:
        product_name = category = quantity = count = expiry_date = "N/A"

    if fruits_vegetables_match:
        # Overwrite product fields with fruit/vegetable data if applicable
        product_name = fruits_vegetables_match.group(1).strip()
        category = "Fruit/Vegetable"
        freshness_index = fruits_vegetables_match.group(2).strip()
        shelf_life = fruits_vegetables_match.group(3).strip()
    else:
        freshness_index = shelf_life = "N/A"

    # Insert row into Excel
    sheet.append([product_name, category, quantity, count, expiry_date, freshness_index, shelf_life])

# Save the workbook
workbook.save("product_analysis.xlsx")
print("Data saved to product_analysis.xlsx")


Saving 750X375_2.jpg to 750X375_2 (1).jpg
Analysis Output for the captured frame:
- For packaged products:
  - Product Name: DiSano Pasta
  - Product Category: Pasta
  - Product Quantity: 1
  - Product Count: 1
  - Expiry Date: Not visible in the image

- For fruits and vegetables:
  - Type of fruit/vegetable: Oats
  - Freshness Index: Good (no visible signs of spoilage or decay)
  - Estimated Shelf Life: 6 months (based on the visual appearance and storage conditions)
Data saved to product_analysis.xlsx
